In [1]:
import firebase_admin
from firebase_admin import credentials, firestore, initialize_app, get_apps, get_app
from math import radians, sin, cos, sqrt, atan2
import time
import os

# --- Configuration ---
# Set the maximum distance a driver can be from the passenger's pickup location (in km).
MAX_MATCH_DISTANCE_KM = 5.0

# Ensure these files are in the same directory as this script.
# NOTE: Replace these placeholder file names with your actual, full service account JSON paths
# if they differ from the ones you uploaded.
PASSENGER_DB_CREDENTIALS = 'passenger-ride-app-firebase-adminsdk-fbsvc-1061e4a556.json'
DRIVER_DB_CREDENTIALS = 'rider-ba88e-firebase-adminsdk-fbsvc-57d40ed3f7.json'

# --- Initialize Firebase Apps ---

def initialize_firebase_app(cred_path, name):
    """Initializes a single Firebase app instance."""
    if not os.path.exists(cred_path):
        print(f"Error: Firebase credential file not found at {cred_path}")
        return None
    try:
        cred = credentials.Certificate(cred_path)
        # Check if the app is already initialized to avoid errors during re-runs
        if any(app.name == name for app in get_apps()):
            app = get_app(name)
        else:
            app = initialize_app(cred, name=name)
        
        db_client = firestore.client(app)
        return db_client
    except Exception as e:
        print(f"Error initializing Firebase App '{name}': {e}")
        return None

# 1. Initialize Passenger DB (for ride requests and public requests)
pdb = initialize_firebase_app(PASSENGER_DB_CREDENTIALS, 'passenger_app')

# 2. Initialize Driver DB (for driver locations and status)
ddb = initialize_firebase_app(DRIVER_DB_CREDENTIALS, 'driver_app')

if not pdb or not ddb:
    print("FATAL: One or both database connections failed. Exiting.")
    exit()

print(f"Listening for new requests (Max Distance: {MAX_MATCH_DISTANCE_KM} km)...")

# --- Distance Calculation (Haversine Formula) ---
def haversine(lat1, lon1, lat2, lon2):
    """
    Calculate the great-circle distance between two points on the Earth 
    (specified in decimal degrees) using the Haversine formula.
    """
    R = 6371  # Radius of Earth in kilometers
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

# --- Core Matching Logic ---
def match_driver(ride_request_doc):
    """Finds the best available driver for a given ride request, applying all filters."""
    ride_id = ride_request_doc.id
    ride_data = ride_request_doc.to_dict()

    print(f"\n=======================================================")
    print(f"[MATCHING] Received pending request for Ride ID: {ride_id}")
    
    # 1. Extract Passenger/Request Data and Preferences
    try:
        pickup_loc = ride_data.get('pickup_location')
        rider_id = ride_data.get('riderId')
        
        # Extract filtering preferences (added in main.dart)
        preferred_vehicle_type = ride_data.get('preferred_vehicle_type') # e.g., 'SUV'
        
        # --- Add other preferences for future expansion (like age, gender, etc.) ---
        # preferred_payment = ride_data.get('preferred_payment_method')
        # has_accessibility = ride_data.get('has_accessibility_needs')
        # min_age = ride_data.get('min_driver_age')
        # max_age = ride_data.get('max_driver_age')

        if not pickup_loc or not isinstance(pickup_loc, firestore.GeoPoint):
            raise KeyError("pickup_location is invalid or missing.")
            
        pickup_lat = pickup_loc.latitude
        pickup_lon = pickup_loc.longitude

    except KeyError as e:
        print(f"[ERROR] Ride request document missing crucial key: {e}. Skipping.")
        # Mark as error so it's not processed again
        pdb.collection('ride_requests').document(ride_id).update({'status': 'error_data_missing'})
        return

    print(f"  Pickup Loc: ({pickup_lat:.4f}, {pickup_lon:.4f})")
    print(f"  Preferred Vehicle: {preferred_vehicle_type or 'Any'} (Filtering applied)")


    # 2. Query and Filter Drivers (Database Filtering)
    try:
        # Step 2a: Query for AVAILABLE drivers
        available_drivers_query = ddb.collection('drivers').where('status', '==', 'available')
        
        # Step 2b: Apply Vehicle Type filter
        if preferred_vehicle_type:
             available_drivers_query = available_drivers_query.where('vehicleType', '==', preferred_vehicle_type)

        # Step 2c: Apply other filters here (e.g., driver license status, etc.)
        
        drivers = available_drivers_query.stream()
        
    except Exception as e:
        print(f"[ERROR] Failed to query drivers database: {e}")
        return

    best_driver = None
    min_distance = float('inf')
    driver_count = 0

    # 3. Find Closest Driver (Geospatial Filtering)
    for driver_doc in drivers:
        driver_count += 1
        driver_data = driver_doc.to_dict()
        driver_id = driver_doc.id
        
        # Ensure the driver has a GeoPoint location for distance calculation
        if 'location' not in driver_data or not isinstance(driver_data.get('location'), firestore.GeoPoint):
            print(f"  Warning: Driver {driver_id} is missing a valid GeoPoint location.")
            continue
            
        driver_loc = driver_data['location']
        driver_lat = driver_loc.latitude
        driver_lon = driver_loc.longitude

        # Calculate Haversine distance
        distance = haversine(pickup_lat, pickup_lon, driver_lat, driver_lon)
        
        # Apply proximity filter (MAX_MATCH_DISTANCE_KM)
        if distance <= MAX_MATCH_DISTANCE_KM:
            print(f"  Driver {driver_id} ({driver_data.get('vehicleType', 'N/A')}) distance: {distance:.2f} km (Potential Match)")
            
            # Find the minimum distance
            if distance < min_distance:
                min_distance = distance
                best_driver = {
                    'id': driver_id,
                    'location': driver_loc,
                    'distance': distance,
                    'data': driver_data
                }
        else:
            print(f"  Driver {driver_id} distance: {distance:.2f} km (Too far)")

    print(f"  Total {driver_count} drivers checked matching preference/availability.")

    # 4. Handle Match Result
    if best_driver:
        print(f"\n[SUCCESS] Matched Ride {ride_id} to Driver {best_driver['id']} at {best_driver['distance']:.2f} km.")
        
        driver_id = best_driver['id']
        driver_loc = best_driver['location']
        driver_name = best_driver['data'].get('name', 'Driver X') # Get driver name if available
        
        try:
            # --- Transaction for Atomicity ---
            @firestore.transactional
            def update_ride_and_driver(transaction):
                # A. Update Ride Request (Passenger DB - private)
                ride_ref = pdb.collection('ride_requests').document(ride_id)
                transaction.update(ride_ref, {
                    'status': 'accepted',
                    'driverId': driver_id,
                    'driverName': driver_name,
                    'driverLocation': driver_loc,
                    'matched_timestamp': firestore.SERVER_TIMESTAMP,
                })

                # B. Update Driver Status (Driver DB - driver is now busy)
                driver_ref = ddb.collection('drivers').document(driver_id)
                transaction.update(driver_ref, {
                    'status': 'on_ride',
                    'currentRideId': ride_id,
                })
                
                # C. Create Public Request (Passenger DB - used by the driver app for lookup)
                public_data = {
                    'pickupLocation': ride_data.get('pickup_location'), 
                    'destinationLocation': ride_data.get('destination'),
                    # Using the actual riderId to map to rider profile data if needed
                    'riderUid': rider_id, 
                    'passengerName': 'chans', # Placeholder from your input data
                    'status': 'pending', 
                    'driverId': driver_id,
                    'timestamp': firestore.SERVER_TIMESTAMP,
                }
                # Use ride_id as the document ID for easy lookup
                pdb.collection('public_ride_requests').document(ride_id).set(public_data)

            # Execute the transaction
            update_ride_and_driver(pdb.transaction())
            print(f"  [DB UPDATE] Transaction successful. Ride updated to 'accepted' and Driver updated to 'on_ride'.")

        except Exception as e:
            print(f"[ERROR] Failed to commit database transaction: {e}")
            
    else:
        print(f"\n[NO MATCH] No suitable driver found for Ride {ride_id} meeting all criteria (SUV, available, < 5km).")
        # Update status to 'unmatched' so it doesn't keep checking this request in the listener
        pdb.collection('ride_requests').document(ride_id).update({'status': 'unmatched'})
        print(f"  [DB UPDATE] Ride {ride_id} status updated to 'unmatched'.")

# --- Real-time Listener ---
def on_snapshot(col_snapshot, changes, read_time):
    """Callback function for the real-time listener."""
    for change in changes:
        # Only process documents that have just been added or modified to 'pending'
        if change.type.name in ('ADDED', 'MODIFIED'):
            doc_data = change.document.to_dict()
            if doc_data and doc_data.get('status') == 'pending':
                match_driver(change.document)

def start_listener():
    """Starts the Firestore real-time listener."""
    # Query: Listen only for ride_requests documents where status is exactly 'pending'
    ride_requests_query = pdb.collection('ride_requests').where('status', '==', 'pending')
    
    # Attach the callback function
    ride_requests_query.on_snapshot(on_snapshot)
    print("\n---------------------------------------------------------")
    print("Matcher Server: Listener started. Waiting for new 'pending' requests...")
    print("---------------------------------------------------------")

# --- Main Execution ---
if __name__ == '__main__':
    try:
        start_listener()
        
        # Keep the script running to listen for real-time updates
        while True:
            time.sleep(1)
            
    except KeyboardInterrupt:
        print("\nMatcher Server stopped by user.")
    except Exception as e:
        print(f"\n[FATAL ERROR] Main loop failed: {e}")


ImportError: cannot import name 'get_apps' from 'firebase_admin' (C:\Users\DELL\anaconda3\Lib\site-packages\firebase_admin\__init__.py)

In [2]:
pip install firebase-admin

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import firebase_admin
from firebase_admin import credentials, firestore, initialize_app
from math import radians, sin, cos, sqrt, atan2
import time
import os

# --- Configuration ---
# Set the maximum distance a driver can be from the passenger's pickup location (in km).
MAX_MATCH_DISTANCE_KM = 5.0

# Ensure these files are in the same directory as this script.
# NOTE: Replace these placeholder file names with your actual, full service account JSON paths
# if they differ from the ones you uploaded.
PASSENGER_DB_CREDENTIALS = 'passenger-ride-app-firebase-adminsdk-fbsvc-1061e4a556.json'
DRIVER_DB_CREDENTIALS = 'rider-ba88e-firebase-adminsdk-fbsvc-57d40ed3f7.json'

# --- Initialize Firebase Apps ---

def initialize_firebase_app(cred_path, name):
    """Initializes a single Firebase app instance."""
    if not os.path.exists(cred_path):
        print(f"Error: Firebase credential file not found at {cred_path}")
        return None
    try:
        cred = credentials.Certificate(cred_path)
        
        app = None
        
        # Check if the app is already initialized using firebase_admin.get_app(name)
        # This is the most reliable cross-version way to check for an existing app.
        try:
            # Attempt to retrieve an existing app instance
            app = firebase_admin.get_app(name)
        except ValueError:
            # If a ValueError is raised, the app does not exist, so initialize it.
            app = initialize_app(cred, name=name)
        
        db_client = firestore.client(app)
        print(f"Matcher Server: {name.capitalize()} DB initialized successfully.")
        return db_client
    except Exception as e:
        print(f"Error initializing Firebase App '{name}': {e}")
        return None

# 1. Initialize Passenger DB (for ride requests and public requests)
pdb = initialize_firebase_app(PASSENGER_DB_CREDENTIALS, 'passenger_app')

# 2. Initialize Driver DB (for driver locations and status)
ddb = initialize_firebase_app(DRIVER_DB_CREDENTIALS, 'driver_app')

if not pdb or not ddb:
    print("FATAL: One or both database connections failed. Exiting.")
    exit()

print(f"Listening for new requests (Max Distance: {MAX_MATCH_DISTANCE_KM} km)...")

# --- Distance Calculation (Haversine Formula) ---
def haversine(lat1, lon1, lat2, lon2):
    """
    Calculate the great-circle distance between two points on the Earth 
    (specified in decimal degrees) using the Haversine formula.
    """
    R = 6371  # Radius of Earth in kilometers
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

# --- Core Matching Logic ---
def match_driver(ride_request_doc):
    """Finds the best available driver for a given ride request, applying all filters."""
    ride_id = ride_request_doc.id
    ride_data = ride_request_doc.to_dict()

    print(f"\n=======================================================")
    print(f"[MATCHING] Received pending request for Ride ID: {ride_id}")
    
    # 1. Extract Passenger/Request Data and Preferences
    try:
        pickup_loc = ride_data.get('pickup_location')
        rider_id = ride_data.get('riderId')
        
        # Extract filtering preferences (added in main.dart)
        preferred_vehicle_type = ride_data.get('preferred_vehicle_type') # e.g., 'SUV'
        
        # --- Add other preferences for future expansion (like age, gender, etc.) ---
        # preferred_payment = ride_data.get('preferred_payment_method')
        # has_accessibility = ride_data.get('has_accessibility_needs')
        # min_age = ride_data.get('min_driver_age')
        # max_age = ride_data.get('max_driver_age')

        if not pickup_loc or not isinstance(pickup_loc, firestore.GeoPoint):
            raise KeyError("pickup_location is invalid or missing.")
            
        pickup_lat = pickup_loc.latitude
        pickup_lon = pickup_loc.longitude

    except KeyError as e:
        print(f"[ERROR] Ride request document missing crucial key: {e}. Skipping.")
        # Mark as error so it's not processed again
        if pdb:
            pdb.collection('ride_requests').document(ride_id).update({'status': 'error_data_missing'})
        return

    print(f"  Pickup Loc: ({pickup_lat:.4f}, {pickup_lon:.4f})")
    print(f"  Preferred Vehicle: {preferred_vehicle_type or 'Any'} (Filtering applied)")


    # 2. Query and Filter Drivers (Database Filtering)
    try:
        # Step 2a: Query for AVAILABLE drivers
        available_drivers_query = ddb.collection('drivers').where('status', '==', 'available')
        
        # Step 2b: Apply Vehicle Type filter
        if preferred_vehicle_type:
             available_drivers_query = available_drivers_query.where('vehicleType', '==', preferred_vehicle_type)

        # Step 2c: Apply other filters here (e.g., driver license status, etc.)
        
        drivers = available_drivers_query.stream()
        
    except Exception as e:
        print(f"[ERROR] Failed to query drivers database: {e}")
        return

    best_driver = None
    min_distance = float('inf')
    driver_count = 0

    # 3. Find Closest Driver (Geospatial Filtering)
    for driver_doc in drivers:
        driver_count += 1
        driver_data = driver_doc.to_dict()
        driver_id = driver_doc.id
        
        # Ensure the driver has a GeoPoint location for distance calculation
        if 'location' not in driver_data or not isinstance(driver_data.get('location'), firestore.GeoPoint):
            print(f"  Warning: Driver {driver_id} is missing a valid GeoPoint location.")
            continue
            
        driver_loc = driver_data['location']
        driver_lat = driver_loc.latitude
        driver_lon = driver_loc.longitude

        # Calculate Haversine distance
        distance = haversine(pickup_lat, pickup_lon, driver_lat, driver_lon)
        
        # Apply proximity filter (MAX_MATCH_DISTANCE_KM)
        if distance <= MAX_MATCH_DISTANCE_KM:
            print(f"  Driver {driver_id} ({driver_data.get('vehicleType', 'N/A')}) distance: {distance:.2f} km (Potential Match)")
            
            # Find the minimum distance
            if distance < min_distance:
                min_distance = distance
                best_driver = {
                    'id': driver_id,
                    'location': driver_loc,
                    'distance': distance,
                    'data': driver_data
                }
        else:
            print(f"  Driver {driver_id} distance: {distance:.2f} km (Too far)")

    print(f"  Total {driver_count} drivers checked matching preference/availability.")

    # 4. Handle Match Result
    if best_driver:
        print(f"\n[SUCCESS] Matched Ride {ride_id} to Driver {best_driver['id']} at {best_driver['distance']:.2f} km.")
        
        driver_id = best_driver['id']
        driver_loc = best_driver['location']
        driver_name = best_driver['data'].get('name', 'Driver X') # Get driver name if available
        
        try:
            # --- Transaction for Atomicity ---
            @firestore.transactional
            def update_ride_and_driver(transaction):
                # A. Update Ride Request (Passenger DB - private)
                ride_ref = pdb.collection('ride_requests').document(ride_id)
                transaction.update(ride_ref, {
                    'status': 'accepted',
                    'driverId': driver_id,
                    'driverName': driver_name,
                    'driverLocation': driver_loc,
                    'matched_timestamp': firestore.SERVER_TIMESTAMP,
                })

                # B. Update Driver Status (Driver DB - driver is now busy)
                driver_ref = ddb.collection('drivers').document(driver_id)
                transaction.update(driver_ref, {
                    'status': 'on_ride',
                    'currentRideId': ride_id,
                })
                
                # C. Create Public Request (Passenger DB - used by the driver app for lookup)
                public_data = {
                    'pickupLocation': ride_data.get('pickup_location'), 
                    'destinationLocation': ride_data.get('destination'),
                    # Using the actual riderId to map to rider profile data if needed
                    'riderUid': rider_id, 
                    'passengerName': 'chans', # Placeholder from your input data
                    'status': 'pending', 
                    'driverId': driver_id,
                    'timestamp': firestore.SERVER_TIMESTAMP,
                }
                # Use ride_id as the document ID for easy lookup
                pdb.collection('public_ride_requests').document(ride_id).set(public_data)

            # Execute the transaction
            update_ride_and_driver(pdb.transaction())
            print(f"  [DB UPDATE] Transaction successful. Ride updated to 'accepted' and Driver updated to 'on_ride'.")

        except Exception as e:
            print(f"[ERROR] Failed to commit database transaction: {e}")
            
    else:
        print(f"\n[NO MATCH] No suitable driver found for Ride {ride_id} meeting all criteria (SUV, available, < 5km).")
        # Update status to 'unmatched' so it doesn't keep checking this request in the listener
        pdb.collection('ride_requests').document(ride_id).update({'status': 'unmatched'})
        print(f"  [DB UPDATE] Ride {ride_id} status updated to 'unmatched'.")

# --- Real-time Listener ---
def on_snapshot(col_snapshot, changes, read_time):
    """Callback function for the real-time listener."""
    # Ensure pdb is initialized before trying to use it in the listener
    if pdb is None:
        print("[ERROR] Listener called, but Passenger DB client (pdb) is None.")
        return

    for change in changes:
        # Only process documents that have just been added or modified to 'pending'
        if change.type.name in ('ADDED', 'MODIFIED'):
            doc_data = change.document.to_dict()
            if doc_data and doc_data.get('status') == 'pending':
                match_driver(change.document)

def start_listener():
    """Starts the Firestore real-time listener."""
    # Query: Listen only for ride_requests documents where status is exactly 'pending'
    ride_requests_query = pdb.collection('ride_requests').where('status', '==', 'pending')
    
    # Attach the callback function
    ride_requests_query.on_snapshot(on_snapshot)
    print("\n---------------------------------------------------------")
    print("Matcher Server: Listener started. Waiting for new 'pending' requests...")
    print("---------------------------------------------------------")

# --- Main Execution ---
if __name__ == '__main__':
    try:
        # Check if DB clients were successfully initialized before starting the listener
        if pdb and ddb:
            start_listener()
            
            # Keep the script running to listen for real-time updates
            while True:
                time.sleep(1)
        else:
            print("[FATAL] Cannot start main loop: Database initialization failed.")
            
    except KeyboardInterrupt:
        print("\nMatcher Server stopped by user.")
    except Exception as e:
        print(f"\n[FATAL ERROR] Main loop failed: {e}")


Matcher Server: Passenger_app DB initialized successfully.
Matcher Server: Driver_app DB initialized successfully.
Listening for new requests (Max Distance: 5.0 km)...


C:\Users\DELL\anaconda3\Lib\site-packages\google\cloud\firestore_v1\base_collection.py:304: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)



---------------------------------------------------------
Matcher Server: Listener started. Waiting for new 'pending' requests...
---------------------------------------------------------

[MATCHING] Received pending request for Ride ID: ZdWzPqsRZVfOuAZRVikF
  Pickup Loc: (12.9972, 77.5263)
  Preferred Vehicle: Any (Filtering applied)
  Total 0 drivers checked matching preference/availability.

[NO MATCH] No suitable driver found for Ride ZdWzPqsRZVfOuAZRVikF meeting all criteria (SUV, available, < 5km).
  [DB UPDATE] Ride ZdWzPqsRZVfOuAZRVikF status updated to 'unmatched'.

[MATCHING] Received pending request for Ride ID: 1O3YjrRuxvmmdVeutRev
  Pickup Loc: (12.9972, 77.5263)
  Preferred Vehicle: Any (Filtering applied)
  Total 0 drivers checked matching preference/availability.

[NO MATCH] No suitable driver found for Ride 1O3YjrRuxvmmdVeutRev meeting all criteria (SUV, available, < 5km).
  [DB UPDATE] Ride 1O3YjrRuxvmmdVeutRev status updated to 'unmatched'.

[MATCHING] Received pend

In [6]:
import firebase_admin
from firebase_admin import credentials, firestore, initialize_app
import os
from datetime import datetime

# --- Configuration ---
PASSENGER_DB_CREDENTIALS = 'passenger-ride-app-firebase-adminsdk-fbsvc-1061e4a556.json'
DRIVER_DB_CREDENTIALS = 'rider-ba88e-firebase-adminsdk-fbsvc-57d40ed3f7.json'

# --- Utility Functions ---

def initialize_firebase_app(cred_path, name):
    """Initializes a single Firebase app instance using the most compatible method."""
    if not os.path.exists(cred_path):
        print(f"Error: Firebase credential file not found at {cred_path}")
        return None
    try:
        cred = credentials.Certificate(cred_path)
        app = None
        
        # Check if the app is already initialized using firebase_admin.get_app(name)
        try:
            app = firebase_admin.get_app(name)
        except ValueError:
            # If the app does not exist, initialize it.
            app = initialize_app(cred, name=name)
        
        db_client = firestore.client(app)
        print(f"Analysis Server: {name.capitalize()} DB initialized successfully.")
        return db_client
    except Exception as e:
        print(f"Error initializing Firebase App '{name}': {e}")
        return None

def format_geopoint(gp):
    """Formats a Firestore GeoPoint object for display."""
    if isinstance(gp, firestore.GeoPoint):
        return f"[{gp.latitude}° N, {gp.longitude}° E]"
    return "N/A"

def format_timestamp(ts):
    """Formats a Firestore Timestamp object for display."""
    if isinstance(ts, datetime):
        return ts.strftime('%Y-%m-%d %I:%M:%S %p %Z')
    return "N/A"

# --- Data Fetching and Analysis Functions ---

def fetch_and_analyze_driver_data(ddb):
    """
    Fetches all documents from the most likely driver/user collection in the Driver DB.
    Tries common collection names: 'rider', 'riders', 'drivers'.
    """
    print("\n" + "="*50)
    print("        DRIVER DATA (RIDER PROJECT)")
    print("="*50)
    
    # List of collection names to check, prioritized by user input ('rider')
    collection_names_to_check = ['rider', 'riders', 'drivers']
    driver_list = []
    found_collection_name = None

    for collection_name in collection_names_to_check:
        try:
            drivers_ref = ddb.collection(collection_name)
            drivers = drivers_ref.stream()
            driver_list = list(drivers)
            
            if driver_list:
                found_collection_name = collection_name
                print(f"SUCCESS: Driver data found in collection '{found_collection_name}'.")
                break
        except Exception as e:
            print(f"Warning: Could not query collection '{collection_name}': {e}")
    
    if not driver_list:
        print(f"No driver records found in any of the checked collections: {', '.join(collection_names_to_check)}.")
        return

    # Basic Analysis Summary
    available_drivers = sum(1 for d in driver_list if d.to_dict().get('status') == 'available')
    on_ride_drivers = sum(1 for d in driver_list if d.to_dict().get('status') == 'on_ride')
    
    print(f"Total Drivers Registered: {len(driver_list)}")
    print(f"Status Summary: Available ({available_drivers}), On Ride ({on_ride_drivers})\n")

    # Detailed Driver Listing
    for i, doc in enumerate(driver_list, 1):
        data = doc.to_dict()
        print(f"--- Driver {i} (ID: {doc.id}) from '{found_collection_name}' ---")
        print(f"UID: {data.get('uid', 'N/A')}")
        print(f"Name: {data.get('name', 'N/A')}")
        print(f"Status: {data.get('status', 'N/A')}")
        print(f"Vehicle Type: {data.get('vehicleType', 'N/A')}")
        
        # Using currentRouteStart and currentRouteEnd as location data as per your schema
        print(f"Live Location (Start): {format_geopoint(data.get('currentRouteStart'))}")
        print(f"Route Start: {format_geopoint(data.get('currentRouteStart'))}")
        print(f"Route End: {format_geopoint(data.get('currentRouteEnd'))}")
        print(f"Last Active: {format_timestamp(data.get('lastActive'))}")
        print("-" * 25)


def fetch_and_analyze_passenger_data(pdb):
    """
    Fetches documents from both 'ride_requests' (private) and 'public_ride_requests' 
    (public) collections in the Passenger DB.
    """
    print("\n" + "="*50)
    print("        PASSENGER RIDE REQUEST DATA")
    print("="*50)

    # --- 1. Private Ride Requests Data ---
    print("\n--- 1. Private Ride Requests ('ride_requests') ---")
    requests_ref = pdb.collection('ride_requests')
    requests = requests_ref.stream()
    request_list = list(requests)
    
    if not request_list:
        print("No private ride requests found.")
    else:
        # Basic Analysis Summary
        pending_requests = sum(1 for r in request_list if r.to_dict().get('status') == 'pending')
        accepted_requests = sum(1 for r in request_list if r.to_dict().get('status') == 'accepted')
        unmatched_requests = sum(1 for r in request_list if r.to_dict().get('status') == 'unmatched')
        
        print(f"Total Private Requests: {len(request_list)}")
        print(f"Status Summary: Pending ({pending_requests}), Accepted ({accepted_requests}), Unmatched ({unmatched_requests})\n")

        # Detailed Request Listing
        for i, doc in enumerate(request_list, 1):
            data = doc.to_dict()
            print(f"--- Request {i} (ID: {doc.id}) ---")
            print(f"Rider ID: {data.get('riderId', 'N/A')}")
            print(f"Status: {data.get('status', 'N/A')}")
            print(f"Pickup: {format_geopoint(data.get('pickup_location'))}")
            print(f"Destination: {format_geopoint(data.get('destination'))}")
            print(f"Vehicle Preference: {data.get('preferred_vehicle_type', 'Any')}")
            if data.get('driverId'):
                 print(f"Matched Driver: {data['driverId']}")
            print("-" * 25)

    # --- 2. Public Ride Requests Data ---
    print("\n--- 2. Public Ride Requests ('public_ride_requests') ---")
    public_ref = pdb.collection('public_ride_requests')
    public_requests = public_ref.stream()
    public_list = list(public_requests)

    if not public_list:
        print("No public ride requests found.")
        print("Note: This collection is only populated by the Matcher Server upon a successful ride match.")
    else:
        print(f"Total Public Requests (Matched/Active): {len(public_list)}\n")
        
        # Detailed Public Listing (Data used by Driver App)
        for i, doc in enumerate(public_list, 1):
            data = doc.to_dict()
            print(f"--- Public Request {i} (Ride ID: {doc.id}) ---")
            print(f"Rider UID: {data.get('riderUid', 'N/A')}")
            print(f"Matched Driver ID: {data.get('driverId', 'N/A')}")
            print(f"Pickup Location: {format_geopoint(data.get('pickupLocation'))}")
            print(f"Destination Location: {format_geopoint(data.get('destinationLocation'))}")
            print(f"Timestamp: {format_timestamp(data.get('timestamp'))}")
            print("-" * 25)


# --- Main Execution ---
if __name__ == '__main__':
    
    # 1. Initialize Passenger DB
    pdb = initialize_firebase_app(PASSENGER_DB_CREDENTIALS, 'passenger_app')

    # 2. Initialize Driver DB
    ddb = initialize_firebase_app(DRIVER_DB_CREDENTIALS, 'driver_app')

    if pdb and ddb:
        # Fetch and analyze data from the Driver DB
        fetch_and_analyze_driver_data(ddb)
        
        # Fetch and analyze data from the Passenger DB (both private and public collections)
        fetch_and_analyze_passenger_data(pdb)

    else:
        print("\nFATAL: Database initialization failed. Cannot perform analysis.")


Analysis Server: Passenger_app DB initialized successfully.
Analysis Server: Driver_app DB initialized successfully.

        DRIVER DATA (RIDER PROJECT)
SUCCESS: Driver data found in collection 'riders'.
Total Drivers Registered: 3
Status Summary: Available (0), On Ride (0)

--- Driver 1 (ID: KHoVWhQqYOZhRRki0VK9) from 'riders' ---
UID: N/A
Name: N/A
Status: N/A
Vehicle Type: N/A
Live Location (Start): N/A
Route Start: N/A
Route End: N/A
Last Active: N/A
-------------------------
--- Driver 2 (ID: NzCDAATMhkUfoKWfRV4KaQviZOq2) from 'riders' ---
UID: NzCDAATMhkUfoKWfRV4KaQviZOq2
Name: chandan
Status: on_route_to_original_destination
Vehicle Type: gear less bike
Live Location (Start): [12.997205° N, 77.5262598° E]
Route Start: [12.997205° N, 77.5262598° E]
Route End: [12.9278196° N, 77.556621° E]
Last Active: 2025-09-30 02:59:19 PM UTC
-------------------------
--- Driver 3 (ID: a8u9W0osH4bbzvvmPUbCV2pManf2) from 'riders' ---
UID: a8u9W0osH4bbzvvmPUbCV2pManf2
Name: RAJU
Status: on_route

In [ ]:
import firebase_admin
from firebase_admin import credentials, firestore, initialize_app, get_app
from firebase_admin.firestore import GeoPoint
from math import radians, sin, cos, sqrt, atan2
import time
import os

# --- Configuration ---
PASSENGER_DB_CREDENTIALS = 'passenger-ride-app-firebase-adminsdk-fbsvc-1061e4a556.json'
DRIVER_DB_CREDENTIALS = 'rider-ba88e-firebase-adminsdk-fbsvc-57d40ed3f7.json'
MAX_MATCH_DISTANCE_KM = 5.0 # Max radius for driver-rider proximity check
MAX_DESTINATION_DEVIATION_KM = 5.0 # Max distance between rider destination and passenger destination
RIDER_COLLECTION_NAME = 'riders' # Confirmed driver collection name
RIDER_MATCH_STATUS = 'on_route_to_original_destination' # Confirmed status for matching

# --- Utility Functions ---

def initialize_firebase_app(cred_path, name):
    """Initializes a single Firebase app instance."""
    if not os.path.exists(cred_path):
        print(f"Error: Firebase credential file not found at {cred_path}")
        return None
    try:
        cred = credentials.Certificate(cred_path)
        app = None
        try:
            app = firebase_admin.get_app(name)
        except ValueError:
            app = initialize_app(cred, name=name)
        
        db_client = firestore.client(app)
        print(f"Matcher Server: {name.capitalize()} DB initialized successfully.")
        return db_client
    except Exception as e:
        print(f"Error initializing Firebase App '{name}': {e}")
        return None

def calculate_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the distance (Haversine) between two coordinates in kilometers.
    """
    R = 6371  # Radius of Earth in kilometers
    
    lat1_rad = radians(lat1)
    lon1_rad = radians(lon1)
    lat2_rad = radians(lat2)
    lon2_rad = radians(lon2)
    
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad
    
    a = sin(dlat / 2)**2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    return R * c

def calculate_dynamic_pickup(driver_loc, passenger_loc):
    """
    Calculates a dynamic pickup point (midway) only if the passenger is generally 
    'on the way' for the driver.
    
    If the two locations are far apart (e.g., > 5km), we assume the passenger's current 
    location is the only viable spot, as a dynamic meeting point might be too complex 
    without a full routing engine. If they are close, we find the midpoint.
    """
    
    # Check if passenger is close enough to be considered for a dynamic pickup point
    dist = calculate_distance(
        driver_loc.latitude, driver_loc.longitude,
        passenger_loc.latitude, passenger_loc.longitude
    )
    
    # Proxy for "passenger can reach a spot in 5 min" or "is generally on the way"
    # If the current distance is < 2km, calculate a midway point for efficiency/pooling.
    if dist < 2.0:
        mid_lat = (driver_loc.latitude + passenger_loc.latitude) / 2
        mid_lon = (driver_loc.longitude + passenger_loc.longitude) / 2
        
        print(f"  [PICKUP LOGIC] Dynamic midpoint calculated ({dist:.2f} km apart).")
        return GeoPoint(mid_lat, mid_lon)
    else:
        # If they are farther apart, passenger's current location is the primary pick-up.
        print(f"  [PICKUP LOGIC] Passenger location used ({dist:.2f} km apart).")
        return passenger_loc


def check_destination_overlap(driver_dest, passenger_dest):
    """
    Checks if the passenger's destination is near the driver's destination 
    (proxy for "on the way").
    """
    
    # Calculate distance between the two destinations
    dist = calculate_distance(
        driver_dest.latitude, driver_dest.longitude,
        passenger_dest.latitude, passenger_dest.longitude
    )
    
    if dist <= MAX_DESTINATION_DEVIATION_KM:
        print(f"  [OVERLAP CHECK] Destination overlap confirmed ({dist:.2f} km deviation).")
        return True
    else:
        print(f"  [OVERLAP CHECK] Destination too far apart ({dist:.2f} km deviation).")
        return False


# --- Main Matching Logic ---

def match_driver(pdb, ddb, ride_request_snapshot):
    """
    Attempts to find a driver for a pending ride request using advanced pooling logic.
    """
    ride_id = ride_request_snapshot.id
    request_data = ride_request_snapshot.to_dict()
    
    # Extract passenger details
    passenger_pickup_loc = request_data.get('pickup_location')
    passenger_dest_loc = request_data.get('destination')
    preferred_vehicle = request_data.get('preferred_vehicle_type', 'Any')
    rider_uid = request_data.get('riderId')
    
    if not (passenger_pickup_loc and passenger_dest_loc):
        print(f"[ERROR] Ride {ride_id} is missing location data. Skipping.")
        return

    print("\n" + "="*50)
    print(f"[MATCHING] Received pending request for Ride ID: {ride_id}")
    print(f"  Pickup Loc: ({passenger_pickup_loc.latitude:.4f}, {passenger_pickup_loc.longitude:.4f})")
    print(f"  Destination Loc: ({passenger_dest_loc.latitude:.4f}, {passenger_dest_loc.longitude:.4f})")
    print(f"  Preferred Vehicle: {preferred_vehicle}")
    
    
    # Step 1: Query for drivers who are on a route and might be poolable
    # We use the 'riders' collection and the confirmed status for active drivers.
    available_drivers_query = ddb.collection(RIDER_COLLECTION_NAME).where('status', '==', RIDER_MATCH_STATUS)
    
    potential_drivers = available_drivers_query.stream()
    driver_list = list(potential_drivers)
    
    total_drivers_checked = 0
    best_match = None
    min_pickup_dist = float('inf')

    for driver_doc in driver_list:
        driver_data = driver_doc.to_dict()
        driver_id = driver_doc.id # Document ID (which is the driver's unique ID)
        
        # Check if the driver data has necessary location fields
        driver_current_loc = driver_data.get('currentRouteStart') # Driver's current location (Start of current route)
        driver_dest_loc = driver_data.get('currentRouteEnd')     # Driver's final destination

        if not (driver_current_loc and driver_dest_loc):
            print(f"  Skipping driver {driver_id}: Missing location data in driver document.")
            continue

        total_drivers_checked += 1
        
        # Step 2: Check Destination Overlap (Poolability)
        if not check_destination_overlap(driver_dest_loc, passenger_dest_loc):
            continue

        # Step 3: Check Initial Proximity (Is the driver close enough to pick up the rider?)
        pickup_dist = calculate_distance(
            driver_current_loc.latitude, driver_current_loc.longitude,
            passenger_pickup_loc.latitude, passenger_pickup_loc.longitude
        )
        
        if pickup_dist > MAX_MATCH_DISTANCE_KM:
            print(f"  Skipping driver {driver_id}: Too far for initial pickup ({pickup_dist:.2f} km).")
            continue

        # Step 4: Check Vehicle Type Match (using lower case for safety)
        driver_vehicle_type = driver_data.get('vehicleType', '').lower()
        if preferred_vehicle != 'Any' and preferred_vehicle.lower() not in driver_vehicle_type:
            print(f"  Skipping driver {driver_id}: Vehicle mismatch (Needs: {preferred_vehicle}, Has: {driver_vehicle_type}).")
            continue

        # Step 5: Successful match candidate! Find the best one.
        if pickup_dist < min_pickup_dist:
            min_pickup_dist = pickup_dist
            best_match = {
                'driver_id': driver_id,
                'driver_data': driver_data,
                'driver_loc': driver_current_loc,
                'driver_dest': driver_dest_loc
            }
            print(f"  Found potential match {driver_id} at {pickup_dist:.2f} km.")

    print(f"  Total {total_drivers_checked} active drivers checked for pooling.")


    if best_match:
        # --- MATCH FOUND ---
        driver_id = best_match['driver_id']
        driver_data = best_match['driver_data']
        
        # Step 6: Calculate Dynamic Pickup Point
        final_pickup_loc = calculate_dynamic_pickup(
            best_match['driver_loc'], passenger_pickup_loc
        )

        print(f"\n[MATCH SUCCESS] Ride {ride_id} matched with Driver {driver_id}!")
        print(f"  Final Pickup Point set to: ({final_pickup_loc.latitude:.4f}, {final_pickup_loc.longitude:.4f})")
        
        # --- Perform Atomic Database Updates (Transaction) ---
        @firestore.transactional
        def update_ride_and_driver(transaction, driver_id, ride_id, final_pickup_loc):
            
            # 1. Update Passenger DB ('ride_requests')
            ride_ref = pdb.collection('ride_requests').document(ride_id)
            transaction.update(ride_ref, {
                'status': 'accepted',
                'driverId': driver_id,
                'matched_at': firestore.SERVER_TIMESTAMP,
                'final_pickup_location': final_pickup_loc
            })
            print(f"  [DB UPDATE] Passenger ride {ride_id} updated to 'accepted'.")

            # 2. Update Driver DB ('riders') - Driver is now 'on_ride' or continuing existing route with new passenger
            # Since the driver is already 'on_route_to_original_destination', 
            # we keep the status but add the new passenger info.
            driver_ref = ddb.collection(RIDER_COLLECTION_NAME).document(driver_id)
            
            # In a real pooling app, you'd update the driver's route/destination here.
            # For simplicity, we just confirm they are still active.
            transaction.update(driver_ref, {
                'current_passenger_id': rider_uid,
                'pooled_ride_id': ride_id
            })
            print(f"  [DB UPDATE] Driver {driver_id} updated with pooled ride.")
            
            # 3. Create Public Ride Request (used by Driver App to see pickup details)
            public_ref = pdb.collection('public_ride_requests').document(ride_id)
            transaction.set(public_ref, {
                'rideId': ride_id,
                'driverId': driver_id,
                'riderUid': rider_uid,
                'pickupLocation': final_pickup_loc,
                'destinationLocation': passenger_dest_loc,
                'vehicleType': driver_data.get('vehicleType', 'N/A'),
                'timestamp': firestore.SERVER_TIMESTAMP
            })
            print(f"  [DB UPDATE] Public ride request created for driver visibility.")
            
        # Execute the transaction
        try:
            transaction = ddb.transaction()
            update_ride_and_driver(transaction, driver_id, ride_id, final_pickup_loc)
        except Exception as e:
            print(f"[FATAL ERROR] Transaction failed for ride {ride_id}: {e}")
            
    else:
        # --- NO MATCH FOUND ---
        print(f"\n[NO MATCH] No suitable driver found for Ride {ride_id} meeting pooling criteria.")
        # Status remains 'pending' so the listener can try again later.


def on_snapshot(col_snapshot, changes, read_time):
    """
    Real-time listener callback for new 'pending' ride requests.
    """
    # Filter for new documents (added) or modified documents where status changed to 'pending'
    for change in changes:
        if change.type.name in ('ADDED', 'MODIFIED'):
            data = change.document.to_dict()
            if data and data.get('status') == 'pending':
                match_driver(db_passenger, db_driver, change.document)


# --- Main Execution ---

if __name__ == '__main__':
    
    # 1. Initialize DBs
    db_passenger = initialize_firebase_app(PASSENGER_DB_CREDENTIALS, 'passenger_app')
    db_driver = initialize_firebase_app(DRIVER_DB_CREDENTIALS, 'driver_app')

    if not (db_passenger and db_driver):
        print("\nFATAL: One or both database connections failed. Exiting.")
        exit()

    print(f"Listening for new requests (Max Match Distance: {MAX_MATCH_DISTANCE_KM} km, Max Destination Deviation: {MAX_DESTINATION_DEVIATION_KM} km)...")
    
    # 2. Start Real-Time Listener on Passenger DB
    try:
        # Query for ride requests that are currently 'pending'
        query_pending = db_passenger.collection('ride_requests').where('status', '==', 'pending')
        
        # Attach the listener
        query_pending.on_snapshot(on_snapshot)
        
        print("-" * 55)
        print("Matcher Server: Listener started. Waiting for new 'pending' requests...")
        print("-" * 55)

        # Keep the main thread alive to listen for updates
        while True:
            time.sleep(1) 

    except Exception as e:
        print(f"[FATAL ERROR] Main loop failed: {e}")


Matcher Server: Passenger_app DB initialized successfully.
Matcher Server: Driver_app DB initialized successfully.
Listening for new requests (Max Match Distance: 5.0 km, Max Destination Deviation: 5.0 km)...
-------------------------------------------------------
Matcher Server: Listener started. Waiting for new 'pending' requests...
-------------------------------------------------------

[MATCHING] Received pending request for Ride ID: i1602PHL9Ntk2uoexQf1
  Pickup Loc: (12.9972, 77.5263)
  Destination Loc: (12.9448, 77.5256)
  Preferred Vehicle: Any
  [OVERLAP CHECK] Destination overlap confirmed (3.86 km deviation).
  Found potential match NzCDAATMhkUfoKWfRV4KaQviZOq2 at 0.00 km.
  [OVERLAP CHECK] Destination overlap confirmed (3.86 km deviation).
  Total 2 active drivers checked for pooling.
  [PICKUP LOGIC] Dynamic midpoint calculated (0.00 km apart).

[MATCH SUCCESS] Ride i1602PHL9Ntk2uoexQf1 matched with Driver NzCDAATMhkUfoKWfRV4KaQviZOq2!
  Final Pickup Point set to: (12.997

In [ ]:
import firebase_admin
from firebase_admin import credentials, firestore, initialize_app, get_app
from firebase_admin.firestore import GeoPoint
from math import radians, sin, cos, sqrt, atan2
import time
import os

# --- Configuration ---
PASSENGER_DB_CREDENTIALS = 'passenger-ride-app-firebase-adminsdk-fbsvc-1061e4a556.json'
DRIVER_DB_CREDENTIALS = 'rider-ba88e-firebase-adminsdk-fbsvc-57d40ed3f7.json'

MAX_MATCH_DISTANCE_KM = 5.0 # Max radius for driver-rider proximity check
MAX_DESTINATION_DEVIATION_KM = 5.0 # Max deviation for pooling

# Database Collection and Status Names
RIDER_COLLECTION_NAME = 'riders' # Confirmed driver collection name in Driver DB
RIDER_MATCH_STATUS = 'on_route_to_original_destination' # Driver status eligible for pooling match
PROPOSAL_COLLECTION_NAME = 'driver_proposals' # Collection for sending ride offers to drivers (in Driver DB)

# Ride Request States
STATUS_PENDING = 'pending'
STATUS_PROPOSED = 'proposed_to_driver'
STATUS_ACCEPTED = 'accepted'
STATUS_ON_ROUTE_TO_PICKUP = 'on_route_to_pickup'

# --- Utility Functions ---

def initialize_firebase_app(cred_path, name):
    """Initializes a single Firebase app instance."""
    if not os.path.exists(cred_path):
        print(f"Error: Firebase credential file not found at {cred_path}")
        return None
    try:
        cred = credentials.Certificate(cred_path)
        app = None
        try:
            app = firebase_admin.get_app(name)
        except ValueError:
            app = initialize_app(cred, name=name)
        
        db_client = firestore.client(app)
        print(f"Matcher Server: {name.capitalize()} DB initialized successfully.")
        return db_client
    except Exception as e:
        print(f"Error initializing Firebase App '{name}': {e}")
        return None

def calculate_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the distance (Haversine) between two coordinates in kilometers.
    """
    R = 6371  # Radius of Earth in kilometers
    
    lat1_rad = radians(lat1)
    lon1_rad = radians(lon1)
    lat2_rad = radians(lat2)
    lon2_rad = radians(lon2)
    
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad
    
    a = sin(dlat / 2)**2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    return R * c

def calculate_dynamic_pickup(driver_loc, passenger_loc):
    """
    Calculates a dynamic pickup point (midway) if the passenger is within 2km 
    of the driver's current position (a proxy for 'on the way' or '5 minute reach').
    Otherwise, the passenger's current location is the designated pickup.
    """
    dist = calculate_distance(
        driver_loc.latitude, driver_loc.longitude,
        passenger_loc.latitude, passenger_loc.longitude
    )
    
    if dist < 2.0:
        mid_lat = (driver_loc.latitude + passenger_loc.latitude) / 2
        mid_lon = (driver_loc.longitude + passenger_loc.longitude) / 2
        
        print(f"  [PICKUP LOGIC] Dynamic midpoint calculated ({dist:.2f} km apart).")
        return GeoPoint(mid_lat, mid_lon)
    else:
        print(f"  [PICKUP LOGIC] Passenger location used ({dist:.2f} km apart).")
        return passenger_loc


def check_destination_overlap(driver_dest, passenger_dest):
    """
    Checks if the passenger's destination is near the driver's destination for pooling.
    """
    dist = calculate_distance(
        driver_dest.latitude, driver_dest.longitude,
        passenger_dest.latitude, passenger_dest.longitude
    )
    
    if dist <= MAX_DESTINATION_DEVIATION_KM:
        print(f"  [OVERLAP CHECK] Destination overlap confirmed ({dist:.2f} km deviation).")
        return True
    else:
        print(f"  [OVERLAP CHECK] Destination too far apart ({dist:.2f} km deviation).")
        return False


# --- Negotiation & Finalization Handlers ---

def finalize_match(pdb, ddb, proposal_data):
    """
    Executed when the driver ACCEPTS the proposed ride. Updates all statuses.
    """
    ride_id = proposal_data['rideId']
    driver_id = proposal_data['driverId']
    final_pickup_loc = proposal_data['pickupLocation']
    rider_uid = proposal_data['riderUid']
    passenger_dest_loc = proposal_data['destinationLocation']
    vehicle_type = proposal_data['vehicleType']

    print(f"\n[ACCEPTANCE] Finalizing match for Ride {ride_id} with Driver {driver_id}.")

    try:
        # 1. Update Passenger DB ('ride_requests') to ACCEPTED
        ride_ref = pdb.collection('ride_requests').document(ride_id)
        ride_ref.update({
            'status': STATUS_ACCEPTED, # Passenger sees this status!
            'driverId': driver_id,
            'matched_at': firestore.SERVER_TIMESTAMP,
            'final_pickup_location': final_pickup_loc,
            'driver_name': proposal_data.get('driverName', 'Driver') 
        })
        print(f"  [DB UPDATE] Passenger ride {ride_id} updated to '{STATUS_ACCEPTED}'.")

        # 2. Update Driver DB ('riders') - Driver is now heading to pick up new passenger
        driver_ref = ddb.collection(RIDER_COLLECTION_NAME).document(driver_id)
        driver_ref.update({
            'status': STATUS_ON_ROUTE_TO_PICKUP, # Driver is now dedicated to pickup
            'current_passenger_id': rider_uid,
            'pooled_ride_id': ride_id,
            'next_target_location': final_pickup_loc # Driver's navigation target
        })
        print(f"  [DB UPDATE] Driver {driver_id} status changed to '{STATUS_ON_ROUTE_TO_PICKUP}'.")
        
        # 3. Create Public Ride Request (for driver app to monitor ride status)
        public_ref = pdb.collection('public_ride_requests').document(ride_id)
        public_ref.set({
            'rideId': ride_id,
            'driverId': driver_id,
            'riderUid': rider_uid,
            'pickupLocation': final_pickup_loc,
            'destinationLocation': passenger_dest_loc,
            'vehicleType': vehicle_type,
            'driverName': proposal_data.get('driverName', 'Driver'),
            'timestamp': firestore.SERVER_TIMESTAMP
        })
        print(f"  [DB UPDATE] Public ride request created for driver visibility.")

        # 4. Cleanup: Delete the proposal document
        ddb.collection(PROPOSAL_COLLECTION_NAME).document(ride_id).delete()
        print(f"  [DB UPDATE] Proposal {ride_id} deleted.")
        
    except Exception as e:
        print(f"[FATAL ERROR] Finalization failed for ride {ride_id}: {e}")

def handle_rejection(pdb, ddb, proposal_data):
    """
    Executed when the driver REJECTS the proposed ride. Reverts status for re-matching.
    """
    ride_id = proposal_data['rideId']

    print(f"\n[REJECTION] Driver rejected Ride {ride_id}. Reverting status.")
    
    try:
        # 1. Update Passenger DB ('ride_requests') back to PENDING/UNMATCHED
        ride_ref = pdb.collection('ride_requests').document(ride_id)
        ride_ref.update({'status': STATUS_PENDING})
        print(f"  [DB UPDATE] Passenger ride {ride_id} reverted to '{STATUS_PENDING}'.")

        # 2. Cleanup: Delete the proposal document
        ddb.collection(PROPOSAL_COLLECTION_NAME).document(ride_id).delete()
        print(f"  [DB UPDATE] Proposal {ride_id} deleted.")

        # NOTE: The driver's status remains 'on_route_to_original_destination'
        # so they are immediately eligible for a new proposal.
        
    except Exception as e:
        print(f"[FATAL ERROR] Rejection handling failed for ride {ride_id}: {e}")


# --- Main Matching Logic ---

def match_driver(pdb, ddb, ride_request_snapshot):
    """
    Finds a suitable driver and initiates the negotiation phase (sends proposal).
    """
    ride_id = ride_request_snapshot.id
    request_data = ride_request_snapshot.to_dict()
    
    passenger_pickup_loc = request_data.get('pickup_location')
    passenger_dest_loc = request_data.get('destination')
    preferred_vehicle = request_data.get('preferred_vehicle_type', 'Any')
    rider_uid = request_data.get('riderId')
    
    if not (passenger_pickup_loc and passenger_dest_loc):
        print(f"[ERROR] Ride {ride_id} is missing location data. Skipping.")
        return

    print("\n" + "="*50)
    print(f"[MATCHING] Searching for driver for Ride ID: {ride_id}")
    
    
    # Step 1: Query for drivers eligible for pooling
    available_drivers_query = ddb.collection(RIDER_COLLECTION_NAME).where('status', '==', RIDER_MATCH_STATUS)
    
    potential_drivers = available_drivers_query.stream()
    driver_list = list(potential_drivers)
    
    total_drivers_checked = 0
    best_match = None
    min_pickup_dist = float('inf')

    for driver_doc in driver_list:
        driver_data = driver_doc.to_dict()
        driver_id = driver_doc.id
        
        # Check if the driver data has necessary location fields
        driver_current_loc = driver_data.get('currentRouteStart')
        driver_dest_loc = driver_data.get('currentRouteEnd')
        driver_name = driver_data.get('name', 'Driver')

        if not (driver_current_loc and driver_dest_loc):
            continue

        total_drivers_checked += 1
        
        # Step 2: Check Destination Overlap (Pooling)
        if not check_destination_overlap(driver_dest_loc, passenger_dest_loc):
            continue

        # Step 3: Check Initial Proximity
        pickup_dist = calculate_distance(
            driver_current_loc.latitude, driver_current_loc.longitude,
            passenger_pickup_loc.latitude, passenger_pickup_loc.longitude
        )
        
        if pickup_dist > MAX_MATCH_DISTANCE_KM:
            continue

        # Step 4: Check Vehicle Type Match
        driver_vehicle_type = driver_data.get('vehicleType', '').lower()
        if preferred_vehicle != 'Any' and preferred_vehicle.lower() not in driver_vehicle_type:
            continue

        # Step 5: Successful match candidate! Find the closest one.
        if pickup_dist < min_pickup_dist:
            min_pickup_dist = pickup_dist
            best_match = {
                'driver_id': driver_id,
                'driver_data': driver_data,
                'driver_loc': driver_current_loc,
                'driver_dest': driver_dest_loc,
                'driver_name': driver_name
            }
            print(f"  Found potential match {driver_id} ({driver_name}) at {pickup_dist:.2f} km.")

    print(f"  Total {total_drivers_checked} active drivers checked for pooling.")


    if best_match:
        # --- PROPOSAL PHASE ---
        driver_id = best_match['driver_id']
        driver_data = best_match['driver_data']
        
        # Step 6: Calculate Dynamic Pickup Point
        final_pickup_loc = calculate_dynamic_pickup(
            best_match['driver_loc'], passenger_pickup_loc
        )

        print(f"\n[PROPOSAL] Proposing Ride {ride_id} to Driver {driver_id}...")
        
        try:
            # 1. Update Passenger DB ('ride_requests') status to proposed
            ride_ref = pdb.collection('ride_requests').document(ride_id)
            ride_ref.update({
                'status': STATUS_PROPOSED,
                'matched_driver_id': driver_id, # Temporary field for tracking
                'proposed_at': firestore.SERVER_TIMESTAMP
            })
            print(f"  [DB UPDATE] Passenger ride {ride_id} updated to '{STATUS_PROPOSED}'.")

            # 2. Create PROPOSAL document in Driver DB
            proposal_ref = ddb.collection(PROPOSAL_COLLECTION_NAME).document(ride_id)
            proposal_ref.set({
                'rideId': ride_id,
                'driverId': driver_id,
                'driverName': best_match['driver_name'],
                'riderUid': rider_uid,
                'status': 'pending_acceptance', # Driver App will monitor and change this field
                'pickupLocation': final_pickup_loc,
                'destinationLocation': passenger_dest_loc,
                'vehicleType': driver_data.get('vehicleType', 'Any'),
                'proposal_timestamp': firestore.SERVER_TIMESTAMP
            })
            print(f"  [DB UPDATE] Proposal sent to Driver {driver_id} via '{PROPOSAL_COLLECTION_NAME}'.")
            
        except Exception as e:
            print(f"[FATAL ERROR] Proposal failed for ride {ride_id}: {e}")
            
    else:
        # --- NO MATCH FOUND ---
        print(f"\n[NO MATCH] No suitable driver found for Ride {ride_id} meeting pooling criteria.")
        # Status remains 'pending' so the listener can try again later.


# --- Snapshot Listeners ---

def on_pending_request_snapshot(col_snapshot, changes, read_time):
    """
    Listener 1: Detects new requests with status 'pending' (from the Passenger App).
    """
    for change in changes:
        if change.type.name in ('ADDED', 'MODIFIED'):
            data = change.document.to_dict()
            if data and data.get('status') == STATUS_PENDING:
                match_driver(db_passenger, db_driver, change.document)


def on_proposal_update_snapshot(col_snapshot, changes, read_time):
    """
    Listener 2: Detects driver response (acceptance/rejection) from the Driver App.
    Driver App updates the status field in the PROPOSAL_COLLECTION_NAME.
    """
    for change in changes:
        data = change.document.to_dict()
        
        # Only process changes where a status is explicitly set to accepted or rejected
        if data and data.get('status') == 'accepted':
            finalize_match(db_passenger, db_driver, data)
        elif data and data.get('status') == 'rejected':
            handle_rejection(db_passenger, db_driver, data)


# --- Main Execution ---

if __name__ == '__main__':
    
    # 1. Initialize DBs
    db_passenger = initialize_firebase_app(PASSENGER_DB_CREDENTIALS, 'passenger_app')
    db_driver = initialize_firebase_app(DRIVER_DB_CREDENTIALS, 'driver_app')

    if not (db_passenger and db_driver):
        print("\nFATAL: One or both database connections failed. Exiting.")
        exit()

    print(f"Listening for new requests (Max Match Distance: {MAX_MATCH_DISTANCE_KM} km, Max Destination Deviation: {MAX_DESTINATION_DEVIATION_KM} km)...")
    
    try:
        # Listener 1: Watch for new 'pending' ride requests (Passenger DB)
        query_pending = db_passenger.collection('ride_requests').where('status', '==', STATUS_PENDING)
        query_pending.on_snapshot(on_pending_request_snapshot)
        
        # Listener 2: Watch for driver responses (Driver DB - Proposals collection)
        query_proposals = db_driver.collection(PROPOSAL_COLLECTION_NAME)
        query_proposals.on_snapshot(on_proposal_update_snapshot)
        
        print("-" * 55)
        print(f"Matcher Server: Two listeners started (Pending Requests & Driver Responses).")
        print("-" * 55)

        # Keep the main thread alive to listen for updates
        while True:
            time.sleep(1) 

    except Exception as e:
        print(f"[FATAL ERROR] Main loop failed: {e}")


Matcher Server: Passenger_app DB initialized successfully.
Matcher Server: Driver_app DB initialized successfully.
Listening for new requests (Max Match Distance: 5.0 km, Max Destination Deviation: 5.0 km)...
-------------------------------------------------------
Matcher Server: Two listeners started (Pending Requests & Driver Responses).
-------------------------------------------------------


C:\Users\DELL\anaconda3\Lib\site-packages\google\cloud\firestore_v1\base_collection.py:304: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)


In [ ]:
import firebase_admin
from firebase_admin import credentials, firestore, initialize_app, get_app
from firebase_admin.firestore import GeoPoint
from math import radians, sin, cos, sqrt, atan2
import time
import os
import sys # Added for graceful exit

# --- Configuration (Use Environment Variables in Production) ---
# NOTE: These paths should ideally be loaded from secure environment variables.
PASSENGER_DB_CREDENTIALS = 'passenger-ride-app-firebase-adminsdk-fbsvc-1061e4a556.json'
DRIVER_DB_CREDENTIALS = 'rider-ba88e-firebase-adminsdk-fbsvc-57d40ed3f7.json'

MAX_MATCH_DISTANCE_KM = 5.0             # Max radius for driver-rider pickup proximity check (Haversine)
MAX_DESTINATION_DEVIATION_KM = 5.0      # Max deviation for destination pooling overlap (Haversine)
DYNAMIC_PICKUP_THRESHOLD_KM = 2.0       # If driver is within this range, use a midpoint pickup

# Database Collection and Status Names
RIDER_COLLECTION_NAME = 'riders'        # Collection name for active drivers (in Driver DB)
RIDER_MATCH_STATUS = 'on_route_to_original_destination' # Driver status eligible for pooling match
PROPOSAL_COLLECTION_NAME = 'driver_proposals' # Collection for sending ride offers (in Driver DB)

# Ride Request States (in Passenger DB's 'ride_requests' collection)
STATUS_PENDING = 'pending'
STATUS_PROPOSED = 'proposed_to_driver'
STATUS_ACCEPTED = 'accepted'
STATUS_ON_ROUTE_TO_PICKUP = 'on_route_to_pickup'

# --- Utility Functions ---

def initialize_firebase_app(cred_path, name):
    """Initializes a single Firebase app instance."""
    if not os.path.exists(cred_path):
        print(f"Error: Firebase credential file not found at {cred_path}", file=sys.stderr)
        return None
    try:
        cred = credentials.Certificate(cred_path)
        app = None
        try:
            # Check if the app is already initialized
            app = firebase_admin.get_app(name)
        except ValueError:
            # Initialize if not found
            app = initialize_app(cred, name=name)
        
        db_client = firestore.client(app)
        print(f"Matcher Server: {name.capitalize()} DB initialized successfully.")
        return db_client
    except Exception as e:
        print(f"Error initializing Firebase App '{name}': {e}", file=sys.stderr)
        return None

def calculate_distance(lat1, lon1, lat2, lon2):
    """
    ALGORITHM: HAVERSINE FORMULA
    Calculate the distance (Haversine) between two coordinates in kilometers.
    This is used for all geospatial filtering (pickup proximity, destination overlap).
    """
    R = 6371  # Radius of Earth in kilometers
    
    lat1_rad = radians(lat1)
    lon1_rad = radians(lon1)
    lat2_rad = radians(lat2)
    lon2_rad = radians(lon2)
    
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad
    
    a = sin(dlat / 2)**2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    return R * c

def calculate_dynamic_pickup(driver_loc, passenger_loc):
    """
    Calculates a dynamic pickup point (midway) if the passenger is within the 
    DYNAMIC_PICKUP_THRESHOLD_KM (2km) of the driver's current position, suggesting 
    a minor detour is efficient. Otherwise, the passenger's original pickup location is used.
    """
    dist = calculate_distance(
        driver_loc.latitude, driver_loc.longitude,
        passenger_loc.latitude, passenger_loc.longitude
    )
    
    if dist < DYNAMIC_PICKUP_THRESHOLD_KM:
        # Calculate geometric midpoint
        mid_lat = (driver_loc.latitude + passenger_loc.latitude) / 2
        mid_lon = (driver_loc.longitude + passenger_loc.longitude) / 2
        
        print(f"  [PICKUP LOGIC] Dynamic midpoint calculated ({dist:.2f} km apart).")
        return GeoPoint(mid_lat, mid_lon)
    else:
        print(f"  [PICKUP LOGIC] Passenger location used ({dist:.2f} km apart).")
        return passenger_loc


def check_destination_overlap(driver_dest, passenger_dest):
    """
    Checks if the passenger's destination is near the driver's destination for pooling.
    Uses MAX_DESTINATION_DEVIATION_KM (5.0 km).
    """
    dist = calculate_distance(
        driver_dest.latitude, driver_dest.longitude,
        passenger_dest.latitude, passenger_dest.longitude
    )
    
    if dist <= MAX_DESTINATION_DEVIATION_KM:
        print(f"  [OVERLAP CHECK] Destination overlap confirmed ({dist:.2f} km deviation).")
        return True
    else:
        print(f"  [OVERLAP CHECK] Destination too far apart ({dist:.2f} km deviation).")
        return False


# --- Negotiation & Finalization Handlers ---

def finalize_match(pdb, ddb, proposal_data):
    """
    Executed when the driver ACCEPTS the proposed ride. Updates all statuses across both DBs.
    """
    ride_id = proposal_data['rideId']
    driver_id = proposal_data['driverId']
    final_pickup_loc = proposal_data['pickupLocation']
    rider_uid = proposal_data['riderUid']
    passenger_dest_loc = proposal_data['destinationLocation']
    vehicle_type = proposal_data['vehicleType']

    print(f"\n[ACCEPTANCE] Finalizing match for Ride {ride_id} with Driver {driver_id}.")

    try:
        # 1. Update Passenger DB ('ride_requests') to ACCEPTED
        ride_ref = pdb.collection('ride_requests').document(ride_id)
        ride_ref.update({
            'status': STATUS_ACCEPTED, 
            'driverId': driver_id,
            'matched_at': firestore.SERVER_TIMESTAMP,
            'finalPickupLocation': final_pickup_loc, # Use camelCase for consistency
            'driverName': proposal_data.get('driverName', 'Driver') 
        })
        print(f"  [DB UPDATE] Passenger ride {ride_id} updated to '{STATUS_ACCEPTED}'.")

        # 2. Update Driver DB ('riders') - Driver is now heading to pick up new passenger
        driver_ref = ddb.collection(RIDER_COLLECTION_NAME).document(driver_id)
        driver_ref.update({
            'status': STATUS_ON_ROUTE_TO_PICKUP, # Driver is now dedicated to pickup
            'currentPassengerId': rider_uid, # Use camelCase for consistency
            'pooledRideId': ride_id,         # Use camelCase for consistency
            'nextTargetLocation': final_pickup_loc # Driver's navigation target
        })
        print(f"  [DB UPDATE] Driver {driver_id} status changed to '{STATUS_ON_ROUTE_TO_PICKUP}'.")
        
        # 3. Create Public Ride Request (for driver app to monitor ride status)
        # This provides the necessary details for the driver's app to manage the new passenger.
        public_ref = pdb.collection('public_ride_requests').document(ride_id)
        public_ref.set({
            'rideId': ride_id,
            'driverId': driver_id,
            'riderUid': rider_uid,
            'pickupLocation': final_pickup_loc,
            'destinationLocation': passenger_dest_loc,
            'vehicleType': vehicle_type,
            'driverName': proposal_data.get('driverName', 'Driver'),
            'timestamp': firestore.SERVER_TIMESTAMP
        })
        print(f"  [DB UPDATE] Public ride request created for driver visibility.")

        # 4. Cleanup: Delete the proposal document
        ddb.collection(PROPOSAL_COLLECTION_NAME).document(ride_id).delete()
        print(f"  [DB UPDATE] Proposal {ride_id} deleted.")
        
    except Exception as e:
        print(f"[FATAL ERROR] Finalization failed for ride {ride_id}: {e}", file=sys.stderr)

def handle_rejection(pdb, ddb, proposal_data):
    """
    Executed when the driver REJECTS the proposed ride. Reverts status for re-matching.
    """
    ride_id = proposal_data['rideId']

    print(f"\n[REJECTION] Driver rejected Ride {ride_id}. Reverting status.")
    
    try:
        # 1. Update Passenger DB ('ride_requests') back to PENDING/UNMATCHED
        ride_ref = pdb.collection('ride_requests').document(ride_id)
        ride_ref.update({'status': STATUS_PENDING})
        print(f"  [DB UPDATE] Passenger ride {ride_id} reverted to '{STATUS_PENDING}'.")

        # 2. Cleanup: Delete the proposal document
        ddb.collection(PROPOSAL_COLLECTION_NAME).document(ride_id).delete()
        print(f"  [DB UPDATE] Proposal {ride_id} deleted.")
        
    except Exception as e:
        print(f"[FATAL ERROR] Rejection handling failed for ride {ride_id}: {e}", file=sys.stderr)


# --- Main Matching Logic ---

def match_driver(pdb, ddb, ride_request_snapshot):
    """
    Finds a suitable driver (eligible for pooling) and initiates the negotiation phase.
    """
    ride_id = ride_request_snapshot.id
    request_data = ride_request_snapshot.to_dict()
    
    # Use camelCase keys for consistency with Firestore document structure
    passenger_pickup_loc = request_data.get('pickupLocation')
    passenger_dest_loc = request_data.get('destinationLocation')
    preferred_vehicle = request_data.get('vehiclePreference', 'Any')
    rider_uid = request_data.get('passengerId') # Use passengerId as the unique rider ID
    
    if not (passenger_pickup_loc and passenger_dest_loc):
        print(f"[ERROR] Ride {ride_id} is missing location data. Skipping.")
        return

    print("\n" + "="*50)
    print(f"[MATCHING] Searching for driver for Ride ID: {ride_id}")
    
    
    # Step 1: Query for drivers eligible for pooling (status == RIDER_MATCH_STATUS)
    # Note: Firestore queries for GeoPoints within a radius require Geohashing, 
    # but for simplicity, we stream all eligible drivers and filter using Haversine.
    available_drivers_query = ddb.collection(RIDER_COLLECTION_NAME).where('status', '==', RIDER_MATCH_STATUS)
    
    potential_drivers = available_drivers_query.stream()
    driver_list = list(potential_drivers)
    
    total_drivers_checked = 0
    best_match = None
    min_pickup_dist = float('inf') # Used for Greedy Selection

    for driver_doc in driver_list:
        driver_data = driver_doc.to_dict()
        driver_id = driver_doc.id
        
        # Driver data uses currentRouteStart and currentRouteEnd for their current ride
        driver_current_loc = driver_data.get('currentRouteStart') # Driver's current location/start of detour
        driver_dest_loc = driver_data.get('currentRouteEnd')       # Driver's current passenger destination
        driver_name = driver_data.get('name', 'Driver')

        if not (driver_current_loc and driver_dest_loc):
            continue

        total_drivers_checked += 1
        
        # Step 2: Check Destination Overlap (within MAX_DESTINATION_DEVIATION_KM)
        if not check_destination_overlap(driver_dest_loc, passenger_dest_loc):
            continue

        # Step 3: Check Initial Proximity (Driver to Passenger Pickup within MAX_MATCH_DISTANCE_KM)
        pickup_dist = calculate_distance(
            driver_current_loc.latitude, driver_current_loc.longitude,
            passenger_pickup_loc.latitude, passenger_pickup_loc.longitude
        )
        
        if pickup_dist > MAX_MATCH_DISTANCE_KM:
            print(f"  Skipping {driver_id}: Pickup too far ({pickup_dist:.2f} km).")
            continue

        # Step 4: Check Vehicle Type Match
        driver_vehicle_type = driver_data.get('vehicleType', '').lower()
        if preferred_vehicle != 'Any' and preferred_vehicle.lower() not in driver_vehicle_type.lower():
            print(f"  Skipping {driver_id}: Vehicle preference mismatch.")
            continue

        # Step 5: ALGORITHM: GREEDY SELECTION
        # Choose the driver who offers the shortest initial pickup distance
        if pickup_dist < min_pickup_dist:
            min_pickup_dist = pickup_dist
            best_match = {
                'driver_id': driver_id,
                'driver_data': driver_data,
                'driver_loc': driver_current_loc,
                'driver_dest': driver_dest_loc,
                'driver_name': driver_name
            }
            print(f"  Found best potential match {driver_id} ({driver_name}) at {pickup_dist:.2f} km.")

    print(f"  Total {total_drivers_checked} active drivers checked for pooling.")


    if best_match:
        # --- PROPOSAL PHASE ---
        driver_id = best_match['driver_id']
        driver_data = best_match['driver_data']
        
        # Step 6: Calculate Dynamic Pickup Point (Midpoint or original passenger location)
        final_pickup_loc = calculate_dynamic_pickup(
            best_match['driver_loc'], passenger_pickup_loc
        )

        print(f"\n[PROPOSAL] Proposing Ride {ride_id} to Driver {driver_id}...")
        
        try:
            # 1. Update Passenger DB ('ride_requests') status to proposed
            ride_ref = pdb.collection('ride_requests').document(ride_id)
            ride_ref.update({
                'status': STATUS_PROPOSED,
                'matchedDriverId': driver_id, # Temporary field for tracking
                'proposedAt': firestore.SERVER_TIMESTAMP
            })
            print(f"  [DB UPDATE] Passenger ride {ride_id} updated to '{STATUS_PROPOSED}'.")

            # 2. Create PROPOSAL document in Driver DB (Driver App listens here)
            proposal_ref = ddb.collection(PROPOSAL_COLLECTION_NAME).document(ride_id)
            proposal_ref.set({
                'rideId': ride_id,
                'driverId': driver_id,
                'driverName': best_match['driver_name'],
                'riderUid': rider_uid,
                'status': 'pending_acceptance', # Driver App will monitor and change this field
                'pickupLocation': final_pickup_loc,
                'destinationLocation': passenger_dest_loc,
                'vehicleType': driver_data.get('vehicleType', 'Any'),
                'proposalTimestamp': firestore.SERVER_TIMESTAMP
            })
            print(f"  [DB UPDATE] Proposal sent to Driver {driver_id} via '{PROPOSAL_COLLECTION_NAME}'.")
            
        except Exception as e:
            print(f"[FATAL ERROR] Proposal failed for ride {ride_id}: {e}", file=sys.stderr)
            
    else:
        # --- NO MATCH FOUND ---
        print(f"\n[NO MATCH] No suitable driver found for Ride {ride_id} meeting pooling criteria.")
        # Status remains 'pending' so the listener can try again later (exponential backoff not implemented here).


# --- Snapshot Listeners ---

def on_pending_request_snapshot(col_snapshot, changes, read_time):
    """
    Listener 1: Detects new requests with status 'pending' (from the Passenger App).
    """
    # Process ADDED (new request) and MODIFIED (rejected requests reverted to pending)
    for change in changes:
        if change.type.name in ('ADDED', 'MODIFIED'):
            data = change.document.to_dict()
            # Double-check status inside the listener callback to filter efficiently
            if data and data.get('status') == STATUS_PENDING:
                print(f"[EVENT] New or reverted PENDING request detected: {change.document.id}")
                match_driver(db_passenger, db_driver, change.document)


def on_proposal_update_snapshot(col_snapshot, changes, read_time):
    """
    Listener 2: Detects driver response (acceptance/rejection) from the Driver App.
    Driver App updates the status field in the PROPOSAL_COLLECTION_NAME.
    """
    for change in changes:
        # Only process changes to existing proposals
        if change.type.name == 'MODIFIED':
            data = change.document.to_dict()
            
            # Only process changes where a status is explicitly set to accepted or rejected
            if data and data.get('status') == 'accepted':
                print(f"[EVENT] Driver accepted proposal: {change.document.id}")
                finalize_match(db_passenger, db_driver, data)
            elif data and data.get('status') == 'rejected':
                print(f"[EVENT] Driver rejected proposal: {change.document.id}")
                handle_rejection(db_passenger, db_driver, data)


# --- Main Execution ---

if __name__ == '__main__':
    
    # 1. Initialize DBs
    db_passenger = initialize_firebase_app(PASSENGER_DB_CREDENTIALS, 'passenger_app')
    db_driver = initialize_firebase_app(DRIVER_DB_CREDENTIALS, 'driver_app')

    if not (db_passenger and db_driver):
        print("\nFATAL: One or both database connections failed. Exiting.", file=sys.stderr)
        sys.exit(1)

    print(f"Listening for new requests (Max Match Distance: {MAX_MATCH_DISTANCE_KM} km, Max Destination Deviation: {MAX_DESTINATION_DEVIATION_KM} km)...")
    
    try:
        # Listener 1: Watch for new 'pending' ride requests (Passenger DB)
        query_pending = db_passenger.collection('ride_requests').where('status', '==', STATUS_PENDING)
        # Note: A .limit(10) or similar batching could be added for high-volume scenarios
        query_pending.on_snapshot(on_pending_request_snapshot)
        
        # Listener 2: Watch for driver responses (Driver DB - Proposals collection)
        query_proposals = db_driver.collection(PROPOSAL_COLLECTION_NAME)
        query_proposals.on_snapshot(on_proposal_update_snapshot)
        
        print("-" * 55)
        print(f"Matcher Server: Two listeners started (Pending Requests & Driver Responses).")
        print("Press Ctrl+C to stop the server.")
        print("-" * 55)

        # Keep the main thread alive to listen for updates
        while True:
            time.sleep(1) 

    except KeyboardInterrupt:
        print("\nMatcher Server stopped by user.")
        sys.exit(0)
    except Exception as e:
        print(f"[FATAL ERROR] Main loop failed: {e}", file=sys.stderr)
        sys.exit(1)


Matcher Server: Passenger_app DB initialized successfully.
Matcher Server: Driver_app DB initialized successfully.
Listening for new requests (Max Match Distance: 5.0 km, Max Destination Deviation: 5.0 km)...


C:\Users\DELL\anaconda3\Lib\site-packages\google\cloud\firestore_v1\base_collection.py:304: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)


-------------------------------------------------------
Matcher Server: Two listeners started (Pending Requests & Driver Responses).
Press Ctrl+C to stop the server.
-------------------------------------------------------


In [ ]:
import firebase_admin
from firebase_admin import credentials, firestore, initialize_app, get_app
from firebase_admin.firestore import GeoPoint
from math import radians, sin, cos, sqrt, atan2
import time
import os
import sys # Added for graceful exit

# --- Configuration (Use Environment Variables in Production) ---
# NOTE: These paths should ideally be loaded from secure environment variables.
PASSENGER_DB_CREDENTIALS = 'passenger-ride-app-firebase-adminsdk-fbsvc-1061e4a556.json'
DRIVER_DB_CREDENTIALS = 'rider-ba88e-firebase-adminsdk-fbsvc-57d40ed3f7.json'

MAX_MATCH_DISTANCE_KM = 5.0             # Max radius for driver-rider pickup proximity check (Haversine)
MAX_DESTINATION_DEVIATION_KM = 5.0      # Max deviation for destination pooling overlap (Haversine)
DYNAMIC_PICKUP_THRESHOLD_KM = 2.0       # If driver is within this range, use a midpoint pickup

# Database Collection and Status Names
RIDER_COLLECTION_NAME = 'riders'        # Collection name for active drivers (in Driver DB)
RIDER_MATCH_STATUS = 'on_route_to_original_destination' # Driver status eligible for pooling match
PROPOSAL_COLLECTION_NAME = 'driver_proposals' # Collection for sending ride offers (in Driver DB)

# Ride Request States (in Passenger DB's 'ride_requests' collection)
STATUS_PENDING = 'pending'
STATUS_PROPOSED = 'proposed_to_driver'
STATUS_ACCEPTED = 'accepted'
STATUS_ON_ROUTE_TO_PICKUP = 'on_route_to_pickup'

# --- Utility Functions ---

def initialize_firebase_app(cred_path, name):
    """Initializes a single Firebase app instance."""
    if not os.path.exists(cred_path):
        print(f"Error: Firebase credential file not found at {cred_path}", file=sys.stderr)
        return None
    try:
        cred = credentials.Certificate(cred_path)
        app = None
        try:
            # Check if the app is already initialized
            app = firebase_admin.get_app(name)
        except ValueError:
            # Initialize if not found
            app = initialize_app(cred, name=name)
        
        db_client = firestore.client(app)
        print(f"Matcher Server: {name.capitalize()} DB initialized successfully.")
        return db_client
    except Exception as e:
        print(f"Error initializing Firebase App '{name}': {e}", file=sys.stderr)
        return None

def calculate_distance(lat1, lon1, lat2, lon2):
    """
    ALGORITHM: HAVERSINE FORMULA
    Calculate the distance (Haversine) between two coordinates in kilometers.
    This is used for all geospatial filtering (pickup proximity, destination overlap).
    """
    R = 6371  # Radius of Earth in kilometers
    
    lat1_rad = radians(lat1)
    lon1_rad = radians(lon1)
    lat2_rad = radians(lat2)
    lon2_rad = radians(lon2)
    
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad
    
    a = sin(dlat / 2)**2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    return R * c

def calculate_dynamic_pickup(driver_loc, passenger_loc):
    """
    Calculates a dynamic pickup point (midway) if the passenger is within the 
    DYNAMIC_PICKUP_THRESHOLD_KM (2km) of the driver's current position, suggesting 
    a minor detour is efficient. Otherwise, the passenger's original pickup location is used.
    """
    dist = calculate_distance(
        driver_loc.latitude, driver_loc.longitude,
        passenger_loc.latitude, passenger_loc.longitude
    )
    
    if dist < DYNAMIC_PICKUP_THRESHOLD_KM:
        # Calculate geometric midpoint
        mid_lat = (driver_loc.latitude + passenger_loc.latitude) / 2
        mid_lon = (driver_loc.longitude + passenger_loc.longitude) / 2
        
        print(f"  [PICKUP LOGIC] Dynamic midpoint calculated ({dist:.2f} km apart).")
        return GeoPoint(mid_lat, mid_lon)
    else:
        print(f"  [PICKUP LOGIC] Passenger location used ({dist:.2f} km apart).")
        return passenger_loc


def check_destination_overlap(driver_dest, passenger_dest):
    """
    Checks if the passenger's destination is near the driver's destination for pooling.
    Uses MAX_DESTINATION_DEVIATION_KM (5.0 km).
    """
    dist = calculate_distance(
        driver_dest.latitude, driver_dest.longitude,
        passenger_dest.latitude, passenger_dest.longitude
    )
    
    if dist <= MAX_DESTINATION_DEVIATION_KM:
        print(f"  [OVERLAP CHECK] Destination overlap confirmed ({dist:.2f} km deviation).")
        return True
    else:
        print(f"  [OVERLAP CHECK] Destination too far apart ({dist:.2f} km deviation).")
        return False


# --- Negotiation & Finalization Handlers ---

def finalize_match(pdb, ddb, proposal_data):
    """
    Executed when the driver ACCEPTS the proposed ride. Updates all statuses across both DBs.
    """
    # NOTE: All these fields are required by finalize_match to update the two DBs.
    ride_id = proposal_data['rideId']
    driver_id = proposal_data['driverId']
    final_pickup_loc = proposal_data['pickupLocation']
    rider_uid = proposal_data['riderUid']
    passenger_dest_loc = proposal_data['destinationLocation']
    vehicle_type = proposal_data['vehicleType']

    print(f"\n[ACCEPTANCE] Finalizing match for Ride {ride_id} with Driver {driver_id}.")

    try:
        # 1. Update Passenger DB ('ride_requests') to ACCEPTED
        ride_ref = pdb.collection('ride_requests').document(ride_id)
        ride_ref.update({
            'status': STATUS_ACCEPTED, 
            'driverId': driver_id,
            'matched_at': firestore.SERVER_TIMESTAMP,
            'finalPickupLocation': final_pickup_loc, # Use camelCase for consistency
            'driverName': proposal_data.get('driverName', 'Driver') 
        })
        print(f"  [DB UPDATE] Passenger ride {ride_id} updated to '{STATUS_ACCEPTED}'.")

        # 2. Update Driver DB ('riders') - Driver is now heading to pick up new passenger
        driver_ref = ddb.collection(RIDER_COLLECTION_NAME).document(driver_id)
        driver_ref.update({
            'status': STATUS_ON_ROUTE_TO_PICKUP, # Driver is now dedicated to pickup
            'currentPassengerId': rider_uid, # Use camelCase for consistency
            'pooledRideId': ride_id,          # Use camelCase for consistency
            'nextTargetLocation': final_pickup_loc # Driver's navigation target
        })
        print(f"  [DB UPDATE] Driver {driver_id} status changed to '{STATUS_ON_ROUTE_TO_PICKUP}'.")
        
        # 3. Create Public Ride Request (for driver app to monitor ride status)
        # This provides the necessary details for the driver's app to manage the new passenger.
        public_ref = pdb.collection('public_ride_requests').document(ride_id)
        public_ref.set({
            'rideId': ride_id,
            'driverId': driver_id,
            'riderUid': rider_uid,
            'pickupLocation': final_pickup_loc,
            'destinationLocation': passenger_dest_loc,
            'vehicleType': vehicle_type,
            'driverName': proposal_data.get('driverName', 'Driver'),
            'timestamp': firestore.SERVER_TIMESTAMP
        })
        print(f"  [DB UPDATE] Public ride request created for driver visibility.")

        # 4. Cleanup: Delete the proposal document
        ddb.collection(PROPOSAL_COLLECTION_NAME).document(ride_id).delete()
        print(f"  [DB UPDATE] Proposal {ride_id} deleted.")
        
    except Exception as e:
        print(f"[FATAL ERROR] Finalization failed for ride {ride_id}: {e}", file=sys.stderr)

def handle_rejection(pdb, ddb, proposal_data):
    """
    Executed when the driver REJECTS the proposed ride. Reverts status for re-matching.
    """
    ride_id = proposal_data['rideId']

    print(f"\n[REJECTION] Driver rejected Ride {ride_id}. Reverting status.")
    
    try:
        # 1. Update Passenger DB ('ride_requests') back to PENDING/UNMATCHED
        ride_ref = pdb.collection('ride_requests').document(ride_id)
        ride_ref.update({'status': STATUS_PENDING})
        print(f"  [DB UPDATE] Passenger ride {ride_id} reverted to '{STATUS_PENDING}'.")

        # 2. Cleanup: Delete the proposal document
        ddb.collection(PROPOSAL_COLLECTION_NAME).document(ride_id).delete()
        print(f"  [DB UPDATE] Proposal {ride_id} deleted.")
        
    except Exception as e:
        print(f"[FATAL ERROR] Rejection handling failed for ride {ride_id}: {e}", file=sys.stderr)


# --- Main Matching Logic ---

def match_driver(pdb, ddb, ride_request_snapshot):
    """
    Finds a suitable driver (eligible for pooling) and initiates the negotiation phase.
    """
    ride_id = ride_request_snapshot.id
    request_data = ride_request_snapshot.to_dict()
    
    # Use camelCase keys for consistency with Firestore document structure
    passenger_pickup_loc = request_data.get('pickupLocation')
    passenger_dest_loc = request_data.get('destinationLocation')
    preferred_vehicle = request_data.get('vehiclePreference', 'Any')
    rider_uid = request_data.get('passengerId') # Use passengerId as the unique rider ID
    
    if not (passenger_pickup_loc and passenger_dest_loc):
        print(f"[ERROR] Ride {ride_id} is missing location data. Skipping.")
        return

    print("\n" + "="*50)
    print(f"[MATCHING] Searching for driver for Ride ID: {ride_id}")
    
    
    # Step 1: Query for drivers eligible for pooling (status == RIDER_MATCH_STATUS)
    # Note: Firestore queries for GeoPoints within a radius require Geohashing, 
    # but for simplicity, we stream all eligible drivers and filter using Haversine.
    available_drivers_query = ddb.collection(RIDER_COLLECTION_NAME).where('status', '==', RIDER_MATCH_STATUS)
    
    potential_drivers = available_drivers_query.stream()
    driver_list = list(potential_drivers)
    
    total_drivers_checked = 0
    best_match = None
    min_pickup_dist = float('inf') # Used for Greedy Selection

    for driver_doc in driver_list:
        driver_data = driver_doc.to_dict()
        driver_id = driver_doc.id
        
        # Driver data uses currentRouteStart and currentRouteEnd for their current ride
        driver_current_loc = driver_data.get('currentRouteStart') # Driver's current location/start of detour
        driver_dest_loc = driver_data.get('currentRouteEnd')      # Driver's current passenger destination
        driver_name = driver_data.get('name', 'Driver')

        if not (driver_current_loc and driver_dest_loc):
            continue

        total_drivers_checked += 1
        
        # Step 2: Check Destination Overlap (within MAX_DESTINATION_DEVIATION_KM)
        if not check_destination_overlap(driver_dest_loc, passenger_dest_loc):
            continue

        # Step 3: Check Initial Proximity (Driver to Passenger Pickup within MAX_MATCH_DISTANCE_KM)
        pickup_dist = calculate_distance(
            driver_current_loc.latitude, driver_current_loc.longitude,
            passenger_pickup_loc.latitude, passenger_pickup_loc.longitude
        )
        
        if pickup_dist > MAX_MATCH_DISTANCE_KM:
            print(f"  Skipping {driver_id}: Pickup too far ({pickup_dist:.2f} km).")
            continue

        # Step 4: Check Vehicle Type Match
        driver_vehicle_type = driver_data.get('vehicleType', '').lower()
        if preferred_vehicle != 'Any' and preferred_vehicle.lower() not in driver_vehicle_type.lower():
            print(f"  Skipping {driver_id}: Vehicle preference mismatch.")
            continue

        # Step 5: ALGORITHM: GREEDY SELECTION
        # Choose the driver who offers the shortest initial pickup distance
        if pickup_dist < min_pickup_dist:
            min_pickup_dist = pickup_dist
            best_match = {
                'driver_id': driver_id,
                'driver_data': driver_data,
                'driver_loc': driver_current_loc,
                'driver_dest': driver_dest_loc,
                'driver_name': driver_name
            }
            print(f"  Found best potential match {driver_id} ({driver_name}) at {pickup_dist:.2f} km.")

    print(f"  Total {total_drivers_checked} active drivers checked for pooling.")


    if best_match:
        # --- PROPOSAL PHASE ---
        driver_id = best_match['driver_id']
        driver_data = best_match['driver_data']
        
        # Step 6: Calculate Dynamic Pickup Point (Midpoint or original passenger location)
        final_pickup_loc = calculate_dynamic_pickup(
            best_match['driver_loc'], passenger_pickup_loc
        )

        print(f"\n[PROPOSAL] Proposing Ride {ride_id} to Driver {driver_id}...")
        
        try:
            # 1. Update Passenger DB ('ride_requests') status to proposed
            ride_ref = pdb.collection('ride_requests').document(ride_id)
            ride_ref.update({
                'status': STATUS_PROPOSED,
                'matchedDriverId': driver_id, # Temporary field for tracking
                'proposedAt': firestore.SERVER_TIMESTAMP
            })
            print(f"  [DB UPDATE] Passenger ride {ride_id} updated to '{STATUS_PROPOSED}'.")

            # 2. Create PROPOSAL document in Driver DB (Driver App listens here)
            proposal_ref = ddb.collection(PROPOSAL_COLLECTION_NAME).document(ride_id)
            
            # REVISED: Only include fields strictly necessary for finalize_match and status tracking
            proposal_ref.set({
                'rideId': ride_id,
                'driverId': driver_id,
                'driverName': best_match['driver_name'],
                'riderUid': rider_uid,
                'status': 'pending_acceptance', # Driver App will monitor and change this field
                'pickupLocation': final_pickup_loc,
                'destinationLocation': passenger_dest_loc,
                'vehicleType': driver_data.get('vehicleType', 'Any'),
            })
            print(f"  [DB UPDATE] Proposal sent to Driver {driver_id} via '{PROPOSAL_COLLECTION_NAME}'.")
            
        except Exception as e:
            print(f"[FATAL ERROR] Proposal failed for ride {ride_id}: {e}", file=sys.stderr)
            
    else:
        # --- NO MATCH FOUND ---
        print(f"\n[NO MATCH] No suitable driver found for Ride {ride_id} meeting pooling criteria.")
        # Status remains 'pending' so the listener can try again later (exponential backoff not implemented here).


# --- Snapshot Listeners ---

def on_pending_request_snapshot(col_snapshot, changes, read_time):
    """
    Listener 1: Detects new requests with status 'pending' (from the Passenger App).
    """
    # Process ADDED (new request) and MODIFIED (rejected requests reverted to pending)
    for change in changes:
        if change.type.name in ('ADDED', 'MODIFIED'):
            data = change.document.to_dict()
            # Double-check status inside the listener callback to filter efficiently
            if data and data.get('status') == STATUS_PENDING:
                print(f"[EVENT] New or reverted PENDING request detected: {change.document.id}")
                match_driver(db_passenger, db_driver, change.document)


def on_proposal_update_snapshot(col_snapshot, changes, read_time):
    """
    Listener 2: Detects driver response (acceptance/rejection) from the Driver App.
    Driver App updates the status field in the PROPOSAL_COLLECTION_NAME.
    """
    for change in changes:
        # Only process changes to existing proposals
        if change.type.name == 'MODIFIED':
            data = change.document.to_dict()
            
            # Only process changes where a status is explicitly set to accepted or rejected
            if data and data.get('status') == 'accepted':
                print(f"[EVENT] Driver accepted proposal: {change.document.id}")
                finalize_match(db_passenger, db_driver, data)
            elif data and data.get('status') == 'rejected':
                print(f"[EVENT] Driver rejected proposal: {change.document.id}")
                handle_rejection(db_passenger, db_driver, data)


# --- Main Execution ---

if __name__ == '__main__':
    
    # 1. Initialize DBs
    db_passenger = initialize_firebase_app(PASSENGER_DB_CREDENTIALS, 'passenger_app')
    db_driver = initialize_firebase_app(DRIVER_DB_CREDENTIALS, 'driver_app')

    if not (db_passenger and db_driver):
        print("\nFATAL: One or both database connections failed. Exiting.", file=sys.stderr)
        sys.exit(1)

    print(f"Listening for new requests (Max Match Distance: {MAX_MATCH_DISTANCE_KM} km, Max Destination Deviation: {MAX_DESTINATION_DEVIATION_KM} km)...")
    
    try:
        # Listener 1: Watch for new 'pending' ride requests (Passenger DB)
        query_pending = db_passenger.collection('ride_requests').where('status', '==', STATUS_PENDING)
        # Note: A .limit(10) or similar batching could be added for high-volume scenarios
        query_pending.on_snapshot(on_pending_request_snapshot)
        
        # Listener 2: Watch for driver responses (Driver DB - Proposals collection)
        query_proposals = db_driver.collection(PROPOSAL_COLLECTION_NAME)
        query_proposals.on_snapshot(on_proposal_update_snapshot)
        
        print("-" * 55)
        print(f"Matcher Server: Two listeners started (Pending Requests & Driver Responses).")
        print("Press Ctrl+C to stop the server.")
        print("-" * 55)

        # Keep the main thread alive to listen for updates
        while True:
            time.sleep(1) 

    except KeyboardInterrupt:
        print("\nMatcher Server stopped by user.")
        sys.exit(0)
    except Exception as e:
        print(f"[FATAL ERROR] Main loop failed: {e}", file=sys.stderr)
        sys.exit(1)


Matcher Server: Passenger_app DB initialized successfully.
Matcher Server: Driver_app DB initialized successfully.
Listening for new requests (Max Match Distance: 5.0 km, Max Destination Deviation: 5.0 km)...
-------------------------------------------------------
Matcher Server: Two listeners started (Pending Requests & Driver Responses).
Press Ctrl+C to stop the server.
-------------------------------------------------------


C:\Users\DELL\anaconda3\Lib\site-packages\google\cloud\firestore_v1\base_collection.py:304: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)
